In [1]:
import pandas as pd
from selenium import webdriver

In [2]:
driver = webdriver.Firefox()
driver.get('https://my3.raceresult.com/110552/#0_2C3B48')

table = driver.find_element_by_id("divRRPublish")

In [3]:
txt = table.text
print(txt)

Timing and scoring by Ultra Timing Professionals LLC
AN INDUSTRY LEADER IN RFID TIMING EXCELLENCE
Always Accurate, Always Instant, Always LIVE!
4.1667-mile loop, trail during daylight, road at night
General » Lap Details
back
Search 
  #
Measurement
Lap Split
Rest Time
  1
Guillaume Calmettes
54Laps
1 7:31:30 50:08 09:51  
2 8:31:25 50:03 09:56  
3 9:31:56 50:33 09:26  
4 10:31:34 50:12 09:47  
5 11:31:35 50:13 09:46  
6 12:31:11 49:49 10:10  
7 13:30:52 49:29 10:30  
8 14:30:35 49:12 10:47  
9 15:30:53 49:30 10:29  
10 16:30:32 49:09 10:50  
11 17:31:15 49:53 10:06  
12 18:29:52 48:29 11:30  
13 19:22:40 41:18 18:41  
14 20:25:18 43:55 16:04  
15 21:24:46 43:23 16:36  
16 22:25:32 44:09 15:50  
17 23:25:17 43:55 16:04  
18 24:24:58 43:36 16:23  
19 25:26:57 45:35 14:24  
20 26:27:01 45:39 14:20  
21 27:28:09 46:46 13:13  
22 28:28:51 47:28 12:31  
23 29:28:27 47:05 12:54  
24 30:26:35 45:12 14:47  
25 31:31:40 50:18 09:41  
26 32:30:33 49:11 10:48  
27 33:30:47 49:24 10:35  
28 34:30:

In [4]:
driver.close()

In [5]:
lines = txt.splitlines()

In [6]:
bib = 1
numbers = []
for i, line in enumerate(lines):
    try:
        int(line)      
        numbers.append((i, line.strip()))
    except ValueError:
        pass

In [7]:
bibs = []
for current, later in zip(numbers, numbers[1:]):
    if (int(current[0] + 1 < later[0])): # empty laps in the end
        bibs.append(current)
bibs.append(numbers[-1])

In [8]:
lst = []
for i, pair in enumerate(bibs):
    d = dict()
    d['number'] = pair[1]
    d['name'] = lines[pair[0]+1]
    d['num_laps'] = lines[pair[0]+2]
    try:
        d['laps'] = lines[pair[0]+3:bibs[i+1][0]]
    except:
        d['laps'] = lines[pair[0]+3:]
    lst.append(d)

In [9]:
df = pd.DataFrame.from_dict(lst)

In [10]:
df.num_laps = df.num_laps.str.replace("Laps", "").astype('int64')

In [11]:
# https://stackoverflow.com/a/46983212/9968316
column_to_explode = "laps"
res = (df
       .set_index([x for x in df.columns if x != column_to_explode])[column_to_explode]
       .apply(pd.Series)
       .stack()
       .reset_index())
res = res.rename(columns={
          res.columns[-2]:'exploded_{}_index'.format(column_to_explode),
          res.columns[-1]: '{}_exploded'.format(column_to_explode)})

In [12]:
tmp = res.laps_exploded.str.split(" ", expand = True).dropna(axis=1)
tmp.head()

,0,1,2,3,4,5
0,1,7:31:30,50:08,09:51,,
1,2,8:31:25,50:03,09:56,,
2,3,9:31:56,50:33,09:26,,
3,4,10:31:34,50:12,09:47,,
4,5,11:31:35,50:13,09:46,,


In [13]:
res.head()

,name,num_laps,number,exploded_laps_index,laps_exploded
0,Guillaume Calmettes,54,1,0,1 7:31:30 50:08 09:51
1,Guillaume Calmettes,54,1,1,2 8:31:25 50:03 09:56
2,Guillaume Calmettes,54,1,2,3 9:31:56 50:33 09:26
3,Guillaume Calmettes,54,1,3,4 10:31:34 50:12 09:47
4,Guillaume Calmettes,54,1,4,5 11:31:35 50:13 09:46


In [14]:
res['lap'] = tmp[0]
res['measurement'] = tmp[1]
res['lap_split'] = tmp[2]
res['rest_time'] = tmp[3]

In [15]:
res.drop(["exploded_laps_index", "laps_exploded"], inplace=True, axis=1)

In [16]:
res.head()

,name,num_laps,number,lap,measurement,lap_split,rest_time
0,Guillaume Calmettes,54,1,1,7:31:30,50:08,09:51
1,Guillaume Calmettes,54,1,2,8:31:25,50:03,09:56
2,Guillaume Calmettes,54,1,3,9:31:56,50:33,09:26
3,Guillaume Calmettes,54,1,4,10:31:34,50:12,09:47
4,Guillaume Calmettes,54,1,5,11:31:35,50:13,09:46


In [17]:
res.to_csv("../data/2018_laps.csv", index=False)